This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace. 

In [1]:
#Ajay Joseph Nov 9th 2024
#Real estate personalization 

In [2]:
!pip install pandas

import pandas as pd

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, NonNegativeInt, conint, constr 
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.memory import ConversationSummaryMemory, ConversationBufferMemory, CombinedMemory, ChatMessageHistory
from langchain import LLMChain
from langchain.chains import ConversationChain, RetrievalQA
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import CSVLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

from typing import List, Any, Dict, Optional, Tuple

import json
import os



os.environ["OPENAI_API_KEY"] = "voc-632652831126677340389066ac2c1862e4d4.33543057"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"



Defaulting to user installation because normal site-packages is not writeable


In [3]:
#First, generate 10ish listing based on the prompt alreasdy given, 
#2 Create a vector db schema and populate this listings into this DB
#3Get user preferences and based on these have Ai generate the descriptions base don the user pref
#3update the augmented description



# use this augmented description to generate an image using diffusion model 
# pass this image into the clip model to generate image embeddings 
#use text to search these images and return the listing 
#chunk the DB and with embeddings base don a text search return the listing 
# Very doable 

In [4]:
#First, generate 10ish listing based on the prompt already given, 

In [5]:
example_listing = "\n Neighborhood: Green Oaks \n Price: $800,000 \nBedrooms: 3 \nBathrooms: 2 \nHouse Size: 2,000 sqft\n\nDescription: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.\nNeighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze.\n"

In [6]:
#Init llm openai 
model_name = "gpt-3.5-turbo"
temperature = 0.0
llm = ChatOpenAI(model_name=model_name, temperature=temperature, max_tokens = 4000)

In [7]:
#define class rl listing  and inherit from it and make lisitngs 

class RealEstateListing(BaseModel):
    Neighborhood: str
    Price: str
    Bedrooms: conint(gt=0)
    Bathrooms: conint(gt=0)
    House_Size: str
    Description: str
        
class RealEstateListings(BaseModel):
    Listings: List[RealEstateListing]
   

parser = PydanticOutputParser(pydantic_object=RealEstateListings)
#Importnat to convert from json to python 
print(parser.get_format_instructions())


The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"Listings": {"title": "Listings", "type": "array", "items": {"$ref": "#/definitions/RealEstateListing"}}}, "required": ["Listings"], "definitions": {"RealEstateListing": {"title": "RealEstateListing", "type": "object", "properties": {"Neighborhood": {"title": "Neighborhood", "type": "string"}, "Price": {"title": "Price", "type": "string"}, "Bedrooms": {"title": "Bedrooms", "exclusiveMinimum": 0, "type": "integer"}, "Bathrooms": {"title": "Bathrooms", "exclusiveMinimum": 0, "type": "integer"}, "House_Size": {"title": "House Size

In [8]:
#prompt template, look at partial var - using pydantic 
prompt = PromptTemplate(
    template="Context:{context}\n{format_instructions}\n{question}",
    input_variables=[ "context", "question"],
    partial_variables={"format_instructions": parser.get_format_instructions},
)
no_listings = "10"
context = example_listing 
question = f"""\
Using the example real-estate listing given above in the Context please generate JSON Listings without initial numbers in them
with different Neighborhood, output the same keys as the example these are Neighborhood, Price, 
Bedrooms, Bathrooms, House Size, and Description, {no_listings} listings
"""

    
query = prompt.format(context = context, question = question)
print("Query\n", query)

output = llm.predict(query)
print("Output \n", output)

#Store in file
with open("realtor_listings.json","w") as file:
    json.dump(output, file, indent=4) #pretty-printing 



Query
 Context:
 Neighborhood: Green Oaks 
 Price: $800,000 
Bedrooms: 3 
Bathrooms: 2 
House Size: 2,000 sqft

Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.
Neighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze.

The o

In [9]:
result = parser.parse(output)
print(len(result.Listings))


10


In [10]:
print(result.Listings)

[RealEstateListing(Neighborhood='Green Acres', Price='$750,000', Bedrooms=4, Bathrooms=3, House_Size='2,500 sqft', Description='Discover this spacious and modern home in the tranquil Green Acres neighborhood. With a large backyard and updated features, this property is perfect for a growing family.'), RealEstateListing(Neighborhood='Pine Grove', Price='$825,000', Bedrooms=3, Bathrooms=2, House_Size='2,200 sqft', Description='Welcome to Pine Grove, where this cozy home offers a blend of comfort and style. Enjoy the peaceful surroundings and convenient location.'), RealEstateListing(Neighborhood='Maple Ridge', Price='$790,000', Bedrooms=3, Bathrooms=2, House_Size='2,100 sqft', Description='Nestled in the charming Maple Ridge neighborhood, this home features a modern design and eco-friendly amenities. Embrace sustainable living in style.'), RealEstateListing(Neighborhood='Oakwood Heights', Price='$780,000', Bedrooms=3, Bathrooms=2, House_Size='2,150 sqft', Description='Experience the beau

In [11]:
type(output)

str

In [12]:
# Convert from Output strings from LLM to CSV
#First LLM outputs to JSOn
#then json converts to dictionaries usinging parser.parse)"
#use the dict to PD and then to CSV
#I'll also try next with Pydantic to stremline better in option 2 

dict_list=[]
for i in range(len(result.Listings)):
    dict_list.append(result.Listings[i].dict())
    
print(dict_list)

df = pd.DataFrame(dict_list)
df = df.reset_index(drop=True)


df.to_csv("realtor_listings.csv", index=False)

[{'Neighborhood': 'Green Acres', 'Price': '$750,000', 'Bedrooms': 4, 'Bathrooms': 3, 'House_Size': '2,500 sqft', 'Description': 'Discover this spacious and modern home in the tranquil Green Acres neighborhood. With a large backyard and updated features, this property is perfect for a growing family.'}, {'Neighborhood': 'Pine Grove', 'Price': '$825,000', 'Bedrooms': 3, 'Bathrooms': 2, 'House_Size': '2,200 sqft', 'Description': 'Welcome to Pine Grove, where this cozy home offers a blend of comfort and style. Enjoy the peaceful surroundings and convenient location.'}, {'Neighborhood': 'Maple Ridge', 'Price': '$790,000', 'Bedrooms': 3, 'Bathrooms': 2, 'House_Size': '2,100 sqft', 'Description': 'Nestled in the charming Maple Ridge neighborhood, this home features a modern design and eco-friendly amenities. Embrace sustainable living in style.'}, {'Neighborhood': 'Oakwood Heights', 'Price': '$780,000', 'Bedrooms': 3, 'Bathrooms': 2, 'House_Size': '2,150 sqft', 'Description': 'Experience the 

In [13]:
#2 Create a vector db schema and populate this listings into this DB
#Load the CSV listings doc 
loader = CSVLoader("realtor_listings.csv")
docs=loader.load()
len(docs)
print(docs)

[Document(page_content='Neighborhood: Green Acres\nPrice: $750,000\nBedrooms: 4\nBathrooms: 3\nHouse_Size: 2,500 sqft\nDescription: Discover this spacious and modern home in the tranquil Green Acres neighborhood. With a large backyard and updated features, this property is perfect for a growing family.', metadata={'source': 'realtor_listings.csv', 'row': 0}), Document(page_content='Neighborhood: Pine Grove\nPrice: $825,000\nBedrooms: 3\nBathrooms: 2\nHouse_Size: 2,200 sqft\nDescription: Welcome to Pine Grove, where this cozy home offers a blend of comfort and style. Enjoy the peaceful surroundings and convenient location.', metadata={'source': 'realtor_listings.csv', 'row': 1}), Document(page_content='Neighborhood: Maple Ridge\nPrice: $790,000\nBedrooms: 3\nBathrooms: 2\nHouse_Size: 2,100 sqft\nDescription: Nestled in the charming Maple Ridge neighborhood, this home features a modern design and eco-friendly amenities. Embrace sustainable living in style.', metadata={'source': 'realtor_

In [14]:
# Initialize the Text Splitter
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)  
# Adjust chunk_size and chunk_overlap as needed

# Split the loaded documents into chunks
chunks = text_splitter.split_documents(docs)

# Check the number of chunks created
print(f"Number of chunks created: {len(chunks)}")
print(chunks)

# initialize your embeddings model
embeddings = OpenAIEmbeddings()

# populate your vector database with the chunks

db = Chroma(embedding_function=embeddings)
db.add_documents(chunks)


Number of chunks created: 10
[Document(page_content='Neighborhood: Green Acres\nPrice: $750,000\nBedrooms: 4\nBathrooms: 3\nHouse_Size: 2,500 sqft\nDescription: Discover this spacious and modern home in the tranquil Green Acres neighborhood. With a large backyard and updated features, this property is perfect for a growing family.', metadata={'source': 'realtor_listings.csv', 'row': 0}), Document(page_content='Neighborhood: Pine Grove\nPrice: $825,000\nBedrooms: 3\nBathrooms: 2\nHouse_Size: 2,200 sqft\nDescription: Welcome to Pine Grove, where this cozy home offers a blend of comfort and style. Enjoy the peaceful surroundings and convenient location.', metadata={'source': 'realtor_listings.csv', 'row': 1}), Document(page_content='Neighborhood: Maple Ridge\nPrice: $790,000\nBedrooms: 3\nBathrooms: 2\nHouse_Size: 2,100 sqft\nDescription: Nestled in the charming Maple Ridge neighborhood, this home features a modern design and eco-friendly amenities. Embrace sustainable living in style.', 

['52772dee-9fca-11ef-a966-ba197b0378d2',
 '52772e98-9fca-11ef-a966-ba197b0378d2',
 '52772ed4-9fca-11ef-a966-ba197b0378d2',
 '52772ef2-9fca-11ef-a966-ba197b0378d2',
 '52772f10-9fca-11ef-a966-ba197b0378d2',
 '52772f24-9fca-11ef-a966-ba197b0378d2',
 '52772f74-9fca-11ef-a966-ba197b0378d2',
 '52772f92-9fca-11ef-a966-ba197b0378d2',
 '52772fa6-9fca-11ef-a966-ba197b0378d2',
 '52772fc4-9fca-11ef-a966-ba197b0378d2']

In [15]:
#. Check based on this basic query. a best. match on whats in the DB 
query = """
    Based on the below  context on houses available , Do you have any 3 bedrooms houseswith atleast 2 bathrooms available?
    """

use_chain_helper = True
if use_chain_helper:
    rag = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=db.as_retriever())
    print(rag.run(query))
else:
    similar_docs = db.similarity_search(query, k=3)
    prompt = PromptTemplate(
        template="{query}\nContext: {context}",
        input_variables=["query", "context"],
    )
    chain = load_qa_chain(llm, prompt = prompt, chain_type="stuff")
    print(chain.run(input_documents=similar_docs, query = query))

Yes, based on the information provided, there are three houses with 3 bedrooms and at least 2 bathrooms available. These are located in the neighborhoods of Hickory Hills, Pine Grove, and Oakwood Heights.


In [16]:
#3Get user preferences and based on these have Ai 
#generate the descriptions based on the user pref
#List of Q & A 
""""
Collect buyer preferences, such as the number of bedrooms, bathrooms, 
location, and other specific requirements from a set of questions or 
telling the buyer to enter their preferences in natural language. 
You can hard-code the buyer preferences in questions and answers, 
or collect them interactively however you'd like, example:
"""

'"\nCollect buyer preferences, such as the number of bedrooms, bathrooms, \nlocation, and other specific requirements from a set of questions or \ntelling the buyer to enter their preferences in natural language. \nYou can hard-code the buyer preferences in questions and answers, \nor collect them interactively however you\'d like, example:\n'

In [17]:
buyer_questions = [   
                "How big do you want your house to be?",
                "What are 3 most important things for you in choosing this property?", 
                "Which amenities would you like?", 
                "Which transportation options are important to you?",
                "How urban do you want your neighborhood to be?",   
            ]
buyer_answers = [
    "A comfortable four-bedroom house atleast 2 bathrooms. with a spacious kitchen with island, a cozy living room, fireplace and gym-room.",
    "A quiet neighborhood, good local schools, proximity to church and temple and convenient shopping options.",
    "A backyard for gardening,  a two-car garage, and a modern, energy-efficient heating system.",
    "Easy access to a train station, bus-station, , proximity to a major highway, and bike-friendly roads.",
    "Access to the walking trails are important, a balance between suburban tranquility and access to urban amenities like restaurants and theaters."
]

In [18]:
#prime chathistory 
history = ChatMessageHistory()

history.add_user_message(f"""You are AI that will personalize and recommend to the user 
ie home buyer a house. This recommendation will be based on their 
answers to personal questions you ask.These questions will 
collect buyer preferences, such as the number of bedrooms, bathrooms, 
location, price and other specific requirements 
from a set of questions AI ask user {len(buyer_questions)} questions""")

for i in range(len(buyer_questions)):
    history.add_ai_message(buyer_questions[i])
    history.add_user_message(buyer_answers[i])

In [19]:
"""
Step 5: Searching Based on Preferences

Semantic Search Implementation: Use the structured buyer preferences to perform a semantic search on the 
vector database, retrieving listings that most closely match the user's requirements.
Listing Retrieval Logic: Fine-tune the retrieval algorithm to ensure that the most relevant listings are 
selected based on the semantic closeness to the buyer’s preferences.
Step 6: Personalizing Listing Descriptions

LLM Augmentation: For each retrieved listing, use the LLM to augment the description, tailoring it to 
resonate with the buyer’s specific preferences. This involves subtly emphasizing aspects of the property 
that align with what the buyer is looking for.
Maintaining Factual Integrity: Ensure that the augmentation process enhances the appeal of the listing 
without altering factual information.

"""

"\nStep 5: Searching Based on Preferences\n\nSemantic Search Implementation: Use the structured buyer preferences to perform a semantic search on the \nvector database, retrieving listings that most closely match the user's requirements.\nListing Retrieval Logic: Fine-tune the retrieval algorithm to ensure that the most relevant listings are \nselected based on the semantic closeness to the buyer’s preferences.\nStep 6: Personalizing Listing Descriptions\n\nLLM Augmentation: For each retrieved listing, use the LLM to augment the description, tailoring it to \nresonate with the buyer’s specific preferences. This involves subtly emphasizing aspects of the property \nthat align with what the buyer is looking for.\nMaintaining Factual Integrity: Ensure that the augmentation process enhances the appeal of the listing \nwithout altering factual information.\n\n"

In [20]:
#first message by ai on what to expect from human 
history.add_ai_message("""Based on house presented to you, personalize your description of the house based on the users 
preferences, after seeing them all, give your recommendation on which house is best for the human based on their need""")


#conversasation summary 
summary_memory = ConversationSummaryMemory(
    llm=llm,
    memory_key="recommendation_summary", 
    input_key="input",
    buffer=f"The human answered {len(buyer_questions)} \
    personal questions). Use them to describe the house for. \
    the buyer, AI should describe its recommendations and how the house fits in \
    with the human preferences described above.",
    return_messages=True)

# you could choose to store some of the q/a in memory as well, in addition to original questions
class MementoBufferMemory(ConversationBufferMemory):
    def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, str]) -> None:
        input_str, output_str = self._get_input_output(inputs, outputs)
        self.chat_memory.add_ai_message(output_str)

#to get ot particular output memory via memory save_context attribuet 
conversational_memory = MementoBufferMemory(
    chat_memory=history,
    memory_key="questions_and_answers", 
    input_key="input"
)

# Combined memory - this is cool is a combo of summary oand ongoing memory 
memory = CombinedMemory(memories=[conversational_memory, summary_memory])


In [21]:
# the first template 

RECOMMENDER_TEMPLATE = \
"""
The following is a friendly conversation between a human and an AI House Recommender. 
For each retrieved listing, use the AI to augment the description, tailoring it to 
resonate with the buyer’s specific preferences. This involves subtly emphasizing aspects of the property 
that align with what the buyer is looking for. Ensure that the 
augmentation process enhances the appeal of the listing without altering factual information.
                        
                        
House for you:
{recommendation_summary}
Personal Questions and Answers:
{questions_and_answers}
Human: {input}
AI:

"""
PROMPT = PromptTemplate(
    input_variables=["recommendation_summary", "input", "questions_and_answers"],
    template=RECOMMENDER_TEMPLATE
)
recommender = ConversationChain(llm=llm, verbose=True, memory=memory, prompt=PROMPT)


In [22]:
#find a best semantic match to the DB based on these answers 
#. Check based on this basic query. a best. match on whats in the DB 
query = """
The following 3 houses match, based on the {buyer_answers} 
"""

similar_docs = db.similarity_search(query, k=3)

#pull out similarity and move to a list 
similar_lists = []

for similar_doc in similar_docs:
    similar_lists.append(similar_doc.page_content)
    
#print(similar_lists)  
    

In [24]:
# Go through the list of houses that match and see what AI recommends on personalization 

for similar_list in similar_lists:
    
    House_recommendation_instructions = \
    f"""
        =====================================
        === START HOUSE RECOMMENDATION ======
        =====================================
        
        House under consideration: 
        
        {similar_list}
        
        \n
        
        RATING INSTRUCTIONS THAT MUST BE STRICTLY FOLLOWED:
        AI will provide a highly personalized recommendation based on the preferences the human answers to questions included with the context. 
        AI should be very sensible to human personal preferences captured in the answers to personal questions, 
        and should not be influenced by anything else.
        AI will also build a persona for human based on human answers to questions, 
        
        OUTPUT FORMAT:
        
        For each retrieved listing, use the AI to augment the description, tailoring it to 
        resonate with the buyer’s specific preferences. This involves subtly emphasizing aspects of the property 
        that align with what the buyer is looking for. Ensure that the augmentation process enhances the appeal of the 
        listing without altering factual information. Also list the original house listing .
    
        FOLLOW THE INSTRUCTIONS STRICTLY, OTHERWISE HUMAN WILL NOT BE ABLE TO UNDERSTAND YOUR REVIEW.
    """
    prediction = recommender.predict(input = House_recommendation_instructions)
    print(prediction)

final_recommendation = \
\
"""Now that AI has reviewed all the houses , AI will recommend human the one that human will like the most and why. 
Personlzie the description and list the original house lisitng, 

"""
prediction = recommender.predict(input=final_recommendation)

print(prediction)



> Entering new ConversationChain chain...
Prompt after formatting:

The following is a friendly conversation between a human and an AI House Recommender. 
For each retrieved listing, use the AI to augment the description, tailoring it to 
resonate with the buyer’s specific preferences. This involves subtly emphasizing aspects of the property 
that align with what the buyer is looking for. Ensure that the 
augmentation process enhances the appeal of the listing without altering factual information.
                        
                        
House for you:
[SystemMessage(content='The human answered 5     personal questions). Use them to describe the house for.     the buyer, AI should describe its recommendations and how the house fits in     with the human preferences described above.')]
Personal Questions and Answers:
Human: You are AI that will personalize and recommend to the user 
ie home buyer a house. This recommendation will be based on their 
answers to personal questio


> Finished chain.
Based on your preferences for a comfortable four-bedroom house with at least 2 bathrooms, a spacious kitchen with an island, a cozy living room, fireplace, and gym-room, I recommend the house in Cedar Hills. This modern home offers 4 bedrooms and 3 bathrooms, providing ample space for your family. The spacious living areas and outdoor space will allow you to enjoy both indoor and outdoor activities. The desirable Cedar Hills neighborhood offers a perfect blend of comfort and convenience, aligning with your desire for a quiet neighborhood with good local schools. The price of $795,000 is slightly above your budget, but the features and location make this house a great option for you.


> Entering new ConversationChain chain...
Prompt after formatting:

The following is a friendly conversation between a human and an AI House Recommender. 
For each retrieved listing, use the AI to augment the description, tailoring it to 
resonate with the buyer’s specific preferences. 


> Finished chain.
Based on your preferences for a comfortable four-bedroom house with at least 2 bathrooms, a spacious kitchen with an island, a cozy living room, fireplace, and gym-room, I recommend the house in Green Acres. This modern and spacious home offers 4 bedrooms and 3 bathrooms, perfect for your family. The large backyard is ideal for gardening, and the updated features align with your desire for a modern, energy-efficient heating system. The tranquil neighborhood and proximity to good local schools make this property a great fit for you. Additionally, the balance between suburban tranquility and access to urban amenities like restaurants and theaters will provide the lifestyle you are looking for. The price of $750,000 is within your budget, making this house a top recommendation for you.

Original House Listing:
House in Green Acres
- 4 bedrooms, 3 bathrooms
- Large backyard for gardening
- Modern and spacious home
- Updated features
- Tranquil neighborhood
- Proximity to

In [ ]:
"""
Example Outputs: Include example outputs showcasing how user 
preferences are processed and how the application generates 
personalized listing descriptions. You can include these in 
comments in your application or in a Jupyter notebook that's 
saved with outputs.
"""



In [ ]:
"""
ROUND1:
buyer_answers = [
    "A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
    "A quiet neighborhood, good local schools, and convenient shopping options.",
    "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
    "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
    "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."
]


OUTPUT1:
    Based on your preferences for a comfortable three-bedroom house 
    with a spacious kitchen and cozy living room, this charming 
    home in Birchwood Heights could be a good fit for you. While 
    it may not have the extra bedroom you desire, the cozy 
    atmosphere and beautifully landscaped yard offer a peaceful 
    environment that aligns with your preference for a quiet 
    neighborhood. The house's 2,050 sqft size provides ample 
    space for your needs, and the price of $770,000 falls 
    within your desired range. If you prioritize a modern and 
    spacious layout, the previously recommended homes in 
    Green Acres or Cedar Hills might better suit your needs.
    
OUTPUT2:
Based on your preferences for a comfortable three-bedroom house 
with a spacious kitchen and cozy living room, this 3-bedroom, 
2-bathroom home in the serene Cedar Hills neighborhood could 
be a good fit for you. While it may not have the extra bedroom 
you desire, the modern amenities and eco-friendly features align 
with your desire for a modern and sustainable living space. The 
tranquil environment of the green community resonates with your 
preference for a quiet neighborhood. However, if having an extra 
bedroom is a priority for you, the previously recommended 
4-bedroom, 3-bathroom home in the Green Acres neighborhood 
might better suit your needs.

OUTPUT3:
Based on your preferences for a comfortable three-bedroom house 
with a spacious kitchen and cozy living room, this modern and 
spacious 4-bedroom, 3-bathroom home in the Green Acres neighborhood 
would be a great fit for you. The large backyard is perfect for 
gardening, and the updated features align with your desire for 
a modern, energy-efficient heating system. The quiet neighborhood 
and good local schools make this property ideal for a growing f
amily like yours. Additionally, the easy access to a reliable 
bus line and proximity to a major highway cater to your transportation 
needs. Overall, this house combines suburban tranquility with 
convenient amenities, making it a top recommendation for you.

FINAL_RECOMMENDATION:

Based on your preferences for a comfortable three-bedroom house with 
a spacious kitchen and cozy living room, I recommend the modern 
and spacious 4-bedroom, 3-bathroom home in the Green Acres 
neighborhood. This house offers a large backyard for gardening, 
updated features for a modern living space, and is located in a 
quiet neighborhood with good local schools. The proximity to 
transportation options like a reliable bus line and a major 
highway make it an ideal fit for your needs.

Original House Listing:
- 4 bedrooms, 3 bathrooms
- Large backyard
- Updated features
- Quiet neighborhood
- Proximity to transportation options

I believe this house combines all the elements you are looking for 
in a home and will provide you with the comfort and convenience 
you desire.

"""

In [ ]:
"""
ROUND2:

buyer_answers = [
    "A comfortable four-bedroom house atleast 2 bathrooms. with a spacious kitchen with island, a cozy living room, fireplace and gym-room.",
    "A quiet neighborhood, good local schools, proximity to church and temple and convenient shopping options.",
    "A backyard for gardening,  a two-car garage, and a modern, energy-efficient heating system.",
    "Easy access to a train station, bus-station, , proximity to a major highway, and bike-friendly roads.",
    "Access to the walking trails are important, a balance between suburban tranquility and access to urban amenities like restaurants and theaters."
]

OUTPUT1:
Based on your preferences for a comfortable four-bedroom house with at least 2 bathrooms, a spacious kitchen 
with an island, a cozy living room, fireplace, and gym-room, I recommend considering this charming 3-bedroom home i
n Eco Haven. While it may have one less bedroom than your ideal, the eco-friendly features of this house align 
perfectly with your desire for sustainable living. The modern amenities and peaceful setting in Eco Haven offer 
a serene environment for you to enjoy. Additionally, the neighborhood's focus on eco-conscious living will 
resonate with your values. This house provides a great opportunity for you to embrace a green lifestyle while 
still enjoying the comforts of home.




OUTPUT2:

Based on your preferences for a comfortable four-bedroom house with at least 2 bathrooms, a spacious kitchen 
with an island, a cozy living room, fireplace, and gym-room, I recommend considering this charming 3-bedroom home 
in Green Haven. While it may have one less bedroom than your ideal, the eco-conscious features and cozy atmosphere 
of this house align well with your desire for a sustainable living environment. Additionally, the neighborhood of 
Green Haven offers a quiet atmosphere, good local schools, and convenient shopping options, making it a great fit 
for your preferences. The house also features a backyard for gardening, a two-car garage, and a modern, 
energy-efficient heating system, meeting your desired amenities. With easy access to walking trails and a 
balance between suburban tranquility and urban amenities like restaurants and theaters, this house in Green Haven 
could be the perfect blend of your ideal home features.


OUTPUT3:

Based on your preferences for a comfortable four-bedroom house with at least 2 bathrooms, a spacious kitchen 
with an island, a cozy living room, fireplace, and gym-room, I recommend considering this charming 3-bedroom home 
in Eco Haven. While it may have one less bedroom than your ideal, the eco-friendly features of this house align 
perfectly with your desire for sustainable living. The modern amenities and peaceful setting in Eco Haven offer 
a serene environment for you to enjoy. Additionally, the neighborhood's focus on eco-conscious living will 
resonate with your values. This house provides a great opportunity for you to embrace a green lifestyle while 
still enjoying the comforts of home.

Original House Listing:
Neighborhood: Eco-Friendly Haven
Price: $770,000
Bedrooms: 3
Bathrooms: 2
House Size: 2,250 sqft
Description: Discover your eco-friendly haven in this 3-bedroom home. Embrace sustainable living with modern comforts and a peaceful atmosphere.



FINAL RECOOMENDATION:

Based on your preferences for a comfortable four-bedroom house with at least 2 bathrooms, a spacious kitchen 
with an island, a cozy living room, fireplace, and gym-room, I recommend considering this charming 3-bedroom home 
in Eco Haven. While it may have one less bedroom than your ideal, the eco-friendly features of this house align 
perfectly with your desire for sustainable living. The modern amenities and peaceful setting in Eco Haven offer a 
serene environment for you to enjoy. Additionally, the neighborhood's focus on eco-conscious living will resonate 
with your values. This house provides a great opportunity for you to embrace a green lifestyle while still 
enjoying the comforts of home.

Original House Listing:
Neighborhood: Eco-Friendly Haven
Price: $770,000
Bedrooms: 3
Bathrooms: 2
House Size: 2,250 sqft
Description: Discover your eco-friendly haven in this 3-bedroom home. Embrace sustainable living with modern comforts and a peaceful atmosphere.


"""

In [ ]:
#Analysis in Round 1 and Round 2 the personlization goals in the buyer_answer have been found in the answers in both 
#Rounds, it would've better to generate the original data with the descriptions so as not to make the AI model generate adjectives on items like
#gym room etc that was not there in the original model 
#in round 2 I see on eof the outputs is repeated twice - perhaps with more data this will get. equalized.
#all else I have learned how to gen data, gen the personalizations and select a house 

In [ ]:
#TBD CLIP 